In [1]:
import pandas as pd
import numpy as np

import nltk
import string
from nltk.tokenize import RegexpTokenizer

from keras.models import Model
from keras.layers import Input, LSTM, Dense

import pickle

Using TensorFlow backend.


In [2]:
def clean(sentence):
    sentence = str(sentence).lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    return tokens

In [3]:
data = pd.read_csv("news-summary/news_summary.csv", encoding="latin1")
data = data.drop(["author", "date", "headlines", "read_more"], axis=1)
data = data.rename(columns={"text":"summary", "ctext":"text"})

data = data[:100]

In [4]:
clean_summaries = []
for entry in data['summary']:
    clean_summaries.append(["<SOS>"] + clean(entry) + ["<EOS>"])
    
clean_text = []
for entry in data['text']:
    clean_text.append(["<SOS>"] + clean(entry) + ["<EOS>"])

In [5]:
all_text_words = sum(clean_text, [])
all_summary_words = sum(clean_summaries, [])

In [6]:
vocab = list(set(all_summary_words + all_text_words))

with(open("vocab.pkl", "wb")) as f:
    pickle.dump(vocab, f)
    
vocab = []
with(open('vocab.pkl', 'rb')) as f:
    vocab = pickle.load(f)
    
text_vectors = []
for text in clean_text:
    vector = []
    for word in text:
        vector.append(vocab.index(word))
    text_vectors.append(vector)

summary_vectors = []
for summary in clean_summaries:
    vector = []
    for word in summary:
        vector.append(vocab.index(word))
    summary_vectors.append(vector)
    
with(open("summary.pkl", "wb")) as f:
    pickle.dump(summary_vectors, f)
    
with(open("text.pkl", "wb")) as f:
    pickle.dump(text_vectors, f)

In [7]:
text_vectors = []
summary_vectors = []
vocab = []
with(open("text.pkl", "rb")) as f:
    text_vectors = pickle.load(f)
with(open("summary.pkl", "rb")) as f:
    summary_vectors = pickle.load(f)
with(open("vocab.pkl", "rb")) as f:
    vocab = pickle.load(f)

In [8]:
# hyperparameters

batch_size = 64
epochs = 10
# latent dimensionality of the encoding space
latent_dim = 256
# need to increase this
num_samples = 1000

In [34]:
input_words = set(all_text_words)
target_words = set(all_summary_words)
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt) for txt in clean_text])
max_decoder_seq_length = max([len(txt) for txt in clean_summaries])

input_token_index = dict([(char, i) for i, char in enumerate(input_words)])
target_token_index = dict([(char, i) for i, char in enumerate(target_words)])

encoder_input_data = np.zeros(
    (len(clean_text), max_encoder_seq_length, num_encoder_tokens),
    dtype=np.float32)
decoder_input_data = np.zeros(
    (len(clean_text), max_decoder_seq_length, num_decoder_tokens),
    dtype=np.float32)
decoder_target_data = np.zeros(
    (len(clean_text), max_decoder_seq_length, num_decoder_tokens),
    dtype=np.float32)

for i, (input_text, target_summary) in enumerate(zip(clean_text, clean_summaries)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_summary):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t-1, target_token_index[char]] = 1.

In [10]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [11]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


### Training

In [12]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 88s 1s/step - loss: 6.9523 - val_loss: 6.9928
Epoch 2/10
80/80 [==============================] - 88s 1s/step - loss: 6.8799 - val_loss: 6.4114
Epoch 3/10
80/80 [==============================] - 89s 1s/step - loss: 6.1334 - val_loss: 6.3427
Epoch 4/10
80/80 [==============================] - 90s 1s/step - loss: 5.9264 - val_loss: 6.3602
Epoch 5/10
80/80 [==============================] - 89s 1s/step - loss: 5.8825 - val_loss: 6.4139
Epoch 6/10
80/80 [==============================] - 88s 1s/step - loss: 5.8644 - val_loss: 6.4261
Epoch 7/10
80/80 [==============================] - 89s 1s/step - loss: 5.8483 - val_loss: 6.4907
Epoch 8/10
80/80 [==============================] - 88s 1s/step - loss: 5.8347 - val_loss: 6.4806
Epoch 9/10
80/80 [==============================] - 88s 1s/step - loss: 5.8153 - val_loss: 6.4959
Epoch 10/10
80/80 [==============================] - 89s 1s/step - loss: 5

C:\Users\rasik\Anaconda3\envs\mlenv\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [28]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [30]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["<SOS>"]] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == "<EOS>" or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [31]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', clean_text[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['<SOS>', 'the', 'daman', 'and', 'diu', 'administration', 'on', 'wednesday', 'withdrew', 'a', 'circular', 'that', 'asked', 'women', 'staff', 'to', 'tie', 'rakhis', 'on', 'male', 'colleagues', 'after', 'the', 'order', 'triggered', 'a', 'backlash', 'from', 'employees', 'and', 'was', 'ripped', 'apart', 'on', 'social', 'media', 'the', 'union', 'territory', 's', 'administration', 'was', 'forced', 'to', 'retreat', 'within', '24', 'hours', 'of', 'issuing', 'the', 'circular', 'that', 'made', 'it', 'compulsory', 'for', 'its', 'staff', 'to', 'celebrate', 'rakshabandhan', 'at', 'workplace', 'it', 'has', 'been', 'decided', 'to', 'celebrate', 'the', 'festival', 'of', 'rakshabandhan', 'on', 'august', '7', 'in', 'this', 'connection', 'all', 'offices', 'departments', 'shall', 'remain', 'open', 'and', 'celebrate', 'the', 'festival', 'collectively', 'at', 'a', 'suitable', 'time', 'wherein', 'all', 'the', 'lady', 'staff', 'shall', 'tie', 'rakhis', 'to', 'their', 'colleagues', 'the', 'or

-
Input sentence: ['<SOS>', 'lashkar', 'e', 'taiba', 's', 'kashmir', 'commander', 'abu', 'dujana', 'was', 'killed', 'in', 'an', 'encounter', 'in', 'a', 'village', 'in', 'pulwama', 'district', 'of', 'jammu', 'and', 'kashmir', 'earlier', 'this', 'week', 'dujana', 'who', 'had', 'managed', 'to', 'give', 'the', 'security', 'forces', 'a', 'slip', 'several', 'times', 'in', 'the', 'past', 'carried', 'a', 'bounty', 'of', 'rs', '15', 'lakh', 'on', 'his', 'head', 'reports', 'say', 'that', 'dujana', 'had', 'come', 'to', 'meet', 'his', 'wife', 'when', 'he', 'was', 'trapped', 'inside', 'a', 'house', 'in', 'hakripora', 'village', 'security', 'officials', 'involved', 'in', 'the', 'encounter', 'tried', 'their', 'best', 'to', 'convince', 'dujana', 'to', 'surrender', 'but', 'he', 'refused', 'reports', 'say', 'according', 'to', 'reports', 'dujana', 'rejected', 'call', 'for', 'surrender', 'from', 'an', 'army', 'officer', 'the', 'army', 'had', 'commissioned', 'a', 'local', 'to', 'start', 'a', 'telephonic', 

-
Input sentence: ['<SOS>', 'an', 'alleged', 'suspect', 'in', 'a', 'kidnapping', 'case', 'was', 'found', 'hanging', 'inside', 'the', 'washroom', 'of', 'the', 'jahangirpuri', 'police', 'station', 'in', 'north', 'delhi', 'on', 'wednesday', 'hours', 'after', 'he', 'was', 'called', 'by', 'the', 'cops', 'for', 'interrogation', 'the', 'police', 'claimed', 'it', 'was', 'an', 'unprovoked', 'suicide', 'and', 'denied', 'use', 'of', 'force', 'an', 'internal', 'inquiry', 'as', 'well', 'as', 'a', 'magisterial', 'probe', 'is', 'in', 'process', 'police', 'said', 'the', 'relatives', 'of', 'the', 'deceased', 'suspect', 'foul', 'play', 'in', 'the', 'death', 'the', 'dead', 'man', 'is', '32', 'year', 'old', 'raj', 'kumar', 'a', 'native', 'of', 'kasganj', 'in', 'up', 'kumar', 'was', 'unmarried', 'and', 'worked', 'as', 'a', 'security', 'guard', 'at', 'a', 'government', 'school', 'in', 'pitampura', 'till', 'a', 'few', 'months', 'ago', 'during', 'his', 'stint', 'at', 'that', 'school', 'he', 'entered', 'into',

-
Input sentence: ['<SOS>', 'two', 'years', 'after', 'a', 'helicopter', 'crash', 'near', 'the', 'bombay', 'high', 'offshore', 'oil', 'field', 'killed', 'two', 'pilots', 'an', 'inquiry', 'by', 'the', 'air', 'accident', 'investigation', 'bureau', 'aaib', 'found', 'that', 'the', 'chopper', 'was', 'flying', 'at', 'a', 'critically', 'low', 'height', 'without', 'the', 'pilots', 'realising', 'it', 'the', 'pawan', 'hans', 'helicopter', 'was', 'on', 'a', 'training', 'sortie', 'on', 'the', 'evening', 'of', 'november', '4', '2015', 'aviation', 'regulator', 'director', 'general', 'of', 'civil', 'aviation', 'dgca', 'had', 'asked', 'aaib', 'to', 'conduct', 'an', 'inquiry', 'the', 'inquiry', 'report', 'which', 'was', 'made', 'public', 'on', 'wednesday', 'said', 'that', 'the', 'main', 'pilot', 'was', 'flying', 'low', 'and', 'that', 'the', 'other', 'did', 'not', 'realise', 'it', 'after', 'taking', 'off', 'the', 'rontappmeyer', 'rytyr', 'helicopter', 'may', 'have', 'faced', 'difficulties', 'as', 'the', 

-
Input sentence: ['<SOS>', 'only', '26', 'malls', 'and', 'commercial', 'centres', 'on', 'mehrauli', 'gurgaon', 'mg', 'road', 'golf', 'course', 'road', 'and', 'sohna', 'road', 'will', 'offer', 'free', 'parking', 'in', 'their', 'basements', 'the', 'municipal', 'corporation', 'of', 'gurugram', 'mcg', 'announced', 'on', 'wednesday', 'the', 'mcg', 'had', 'earlier', 'said', 'it', 'will', 'waive', 'property', 'tax', 'on', 'basements', 'if', 'the', 'same', 'were', 'used', 'to', 'provide', 'free', 'parking', 'soon', 'afterwards', '45', 'sites', 'offered', 'free', 'parking', 'but', 'now', 'some', 'of', 'the', 'owners', 'have', 'agreed', 'to', 'pay', 'the', 'property', 'tax', 'instead', 'of', 'offering', 'free', 'parking', 'service', 'as', 'a', 'result', 'the', 'list', 'has', 'been', 'trimmed', 'to', '26', 'building', 'owners', 'said', 'the', 'revenue', 'from', 'parking', 'is', 'higher', 'than', 'the', 'gains', 'from', 'the', 'property', 'tax', 'waiver', 'the', 'updated', 'list', 'is', 'in', 'th

...

-
Input sentence: ['<SOS>', 'two', 'new', 'venues', 'thiruvananthapuram', 'in', 'kerala', 'and', 'barsapara', 'in', 'assam', 'are', 'in', 'contention', 'to', 'host', 'test', 'matches', 'during', 'the', 'home', 'series', 'against', 'sri', 'lanka', 'as', 'indian', 'cricket', 'team', 'is', 'set', 'to', 'play', 'a', 'record', '23', 'international', 'matches', 'at', 'home', 'between', 'september', 'and', 'december', 'a', 'decision', 'will', 'be', 'taken', 'by', 'bcci', 's', 'tours', 'and', 'fixtures', 'committee', 'in', 'kolkata', 'on', 'tuesday', 'read', 'virat', 'kohli', 'spoilt', 'for', 'choices', 'ahead', 'of', 'second', 'india', 'vs', 'sri', 'lanka', 'test', 'match', 'according', 'to', 'a', 'bcci', 'office', 'bearer', 'indian', 'cricket', 'team', 'will', 'play', 'five', 'odis', 'and', 'three', 't20', 'internationals', 'against', 'australia', 'from', 'september', 'end', 'to', 'mid', 'of', 'october', 'the', 'series', 'will', 'be', 'followed', 'by', 'three', 'odis', 'and', 'as', 'many', '

-
Input sentence: ['<SOS>', 'scores', 'of', 'easyjet', 'passengers', 'were', 'stranded', 'on', 'the', 'greek', 'island', 'of', 'zakynthos', 'also', 'known', 'as', 'zante', 'after', 'their', 'plane', 'developed', 'technical', 'difficulties', 'and', 'a', 'replacement', 'aircraft', 'was', 'prevented', 'from', 'flying', 'in', 'because', 'of', 'mating', 'turtles', 'passengers', 'expressed', 'fury', 'at', 'easyjet', 's', 'lack', 'of', 'communication', 'and', 'incredibly', 'bad', 'handling', 'of', 'the', 'delay', 'which', 'began', 'on', 'sunday', 'afternoon', 'some', 'travellers', 'who', 'did', 'not', 'book', 'on', 'to', 'other', 'flights', 'were', 'only', 'able', 'to', 'make', 'it', 'home', 'on', 'tuesday', 'a', 'fault', 'with', 'a', 'wing', 'flap', 'was', 'announced', 'by', 'the', 'captain', 'soon', 'after', 'passengers', 'boarded', 'the', 'flight', 'to', 'gatwick', 'at', 'around', '3pm', 'on', 'sunday', 'and', 'there', 'was', 'no', 'communication', 'for', 'another', 'hour', 'despite', 'bei

-
Input sentence: ['<SOS>', 'twenty', 'people', 'including', 'six', 'female', 'employees', 'of', 'a', 'multi', 'national', 'company', 'went', 'through', 'a', 'harrowing', 'time', 'on', 'saturday', 'after', 'being', 'stuck', 'in', 'an', 'elevator', 'at', 'sector', '53', 's', 'south', 'point', 'mall', 'for', 'close', 'to', '90', 'minutes', 'the', 'women', 'according', 'to', 'the', 'police', 'were', 'off', 'duty', 'and', 'were', 'returning', 'home', 'after', 'a', 'party', 'when', 'the', 'elevator', 'got', 'stuck', 'between', 'the', 'second', 'and', 'third', 'floors', 'two', 'of', 'the', 'women', 'fell', 'unconscious', 'due', 'to', 'lack', 'of', 'oxygen', 'said', 'the', 'police', 'some', 'people', 'stuck', 'later', 'called', 'the', 'elevator', 'control', 'room', 'but', 'the', 'staff', 'could', 'not', 'rectify', 'the', 'technical', 'fault', 'after', 'which', 'the', 'police', 'were', 'called', 'a', 'police', 'team', 'soon', 'reached', 'the', 'spot', 'and', 'after', 'assessing', 'the', 'situa

-
Input sentence: ['<SOS>', 'new', 'delhi', 'aug', '1', 'pti', 'shops', 'at', 'the', 'domestic', 'terminal', 'of', 'delhi', 'airport', 'have', 'been', 'relocated', 'to', 'provide', 'additional', 'space', 'for', 'seamless', 'passenger', 'movement', 'the', 'government', 'said', 'today', 'a', 'public', 'accounts', 'committee', 'headed', 'by', 'congress', 'leader', 'k', 'v', 'thomas', 'had', 'in', 'its', 'draft', 'report', 'indicted', 'the', 'delhi', 'airport', 'operator', 'about', 'shrinkage', 'of', 'space', 'for', 'passengers', 'because', 'of', 'restaurants', 'retail', 'outlets', 'duty', 'free', 'complexes', 'and', 'executive', 'lounges', 'the', 'committee', 'had', 'said', 'that', 'this', 'could', 'hamper', 'passenger', 'movement', 'incase', 'of', 'a', 'mishap', '<EOS>']
Decoded sentence: thethethethethethethethethethethethethethethethethethethethethethethe
-
Input sentence: ['<SOS>', 'facing', 'pressure', 'from', 'president', 'donald', 'trump', 'to', 'cut', 'the', 'cost', 'of', 'new', '

-
Input sentence: ['<SOS>', 'bored', 'in', 'confines', 'of', 'a', 'sprawling', 'resort', 'in', 'bengaluru', 'the', 'congress', 'flock', 'of', '44', 'mlas', 'is', 'a', 'frustrated', 'lot', 'now', 'following', 'strict', 'orders', 'of', 'the', 'karnataka', 'power', 'minister', 'dk', 'shivakumar', 'the', 'mlas', 'are', 'not', 'allowed', 'to', 'venture', 'out', 'in', 'the', 'city', 'while', 'the', 'minister', 'is', 'presently', 'in', 'new', 'delhi', 'to', 'seek', 'further', 'course', 'of', 'action', '30', 'mlas', 'have', 'been', 'put', 'in', 'charge', 'of', 'the', 'discipline', 'but', 'few', 'defiant', 'mlas', 'did', 'head', 'out', 'in', 'the', 'city', 'for', 'sightseeing', 'shopping', 'to', 'visit', 'temples', 'much', 'to', 'the', 'dismay', 'of', 'those', 'still', 'under', 'curbs', 'the', 'mlas', 'who', 'were', 'allowed', 'to', 'go', 'out', 'are', 'supposedly', 'those', 'the', 'congress', 'is', 'confident', 'of', 'not', 'ditching', 'the', 'party', '<EOS>']
Decoded sentence: thethethethethe